In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv
/kaggle/input/digit-recognizer/sample_submission.csv


In [2]:
import torch
import numpy as np
import torchvision
import pandas as pd

In [3]:
traindata = pd.read_csv('/kaggle/input/digit-recognizer/train.csv')
testdata = pd.read_csv('/kaggle/input/digit-recognizer/test.csv')

In [4]:
traindata = traindata.values
trainlabel = traindata[0:,0]

In [5]:
traindata = traindata[0:,1:].reshape((-1,28,28),order = 'C')
testdata = testdata.values.reshape((-1,28,28),order = 'C')

In [6]:
import torchvision.transforms as transforms
from torch.utils.data import DataLoader, Dataset

In [7]:
train_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.ToTensor(),
])
test_transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor(),
])

In [8]:
class ImgDataset(Dataset):
    def __init__(self, x,y = None, transform =None):
        self.x = x.astype(np.uint8)
        self.y = y
        if y is not None:
            self.y = torch.LongTensor(y)
        self.transform = transform
    def __len__(self):
        return len(self.x)
    def __getitem__(self,index):
        X = self.x[index]
        if self.transform is not None:
            X = self.transform(X)
        if self.y is not None:
            Y = self.y[index]
            return X,Y
        else:
            return X        

In [9]:
train_set = ImgDataset(traindata, trainlabel, train_transform)
batch_size = 128
train_loader = DataLoader(train_set,batch_size = batch_size, shuffle = True)

In [10]:
import torch.nn as nn
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier,self).__init__()
        self.cnn = nn.Sequential(
            nn.Conv2d(1,3,3,1,1),#[3,28,28]
            nn.BatchNorm2d(3),
            nn.ReLU(),
            nn.MaxPool2d(2,2,0),#[3,14，14]
            
            nn.Conv2d(3,64,3,1,2),#[64,16,16]
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2,2,0),#[64,8,8]
            
            nn.Conv2d(64,128,3,1,1),
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2,2,0), #[128,4,4]       
        )
        self.fc = nn.Sequential(
            nn.Linear(128*4*4,256),
            nn.ReLU(),
            nn.Linear(256,128),
            nn.ReLU(),
            nn.Linear(128,10),
    )
    
    def forward(self,x):
        out = self.cnn(x)
        out = out.view(out.size()[0],-1)
        return self.fc(out)

In [11]:
model = Classifier().cuda()
loss = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=0.001)
num_epoch =5

In [12]:
import time
for epoch in range(num_epoch):
    epoch_start_time = time.time()
    train_acc = 0.0
    train_loss = 0.0
    model.train()
    for i,data in enumerate(train_loader):
        optimizer.zero_grad()
        train_pred = model(data[0].cuda())
        batch_loss = loss(train_pred,data[1].cuda())
        batch_loss.backward()
        optimizer.step()
        
        train_acc += np.sum(np.argmax(train_pred.cpu().data.numpy(),axis = 1)== data[1].numpy())
        train_loss += batch_loss.item()
    print('[%03d/%03d] %2.2f sec(s) Train Acc: %3.6f Loss: %3.6f ' % \
            (epoch + 1, num_epoch, time.time()-epoch_start_time, \
             train_acc/train_set.__len__(), train_loss/train_set.__len__()))

        

[001/005] 10.60 sec(s) Train Acc: 0.879738 Loss: 0.002858 
[002/005] 8.79 sec(s) Train Acc: 0.950476 Loss: 0.001200 
[003/005] 8.82 sec(s) Train Acc: 0.963833 Loss: 0.000880 
[004/005] 9.95 sec(s) Train Acc: 0.969381 Loss: 0.000747 
[005/005] 9.14 sec(s) Train Acc: 0.973262 Loss: 0.000655 


In [13]:
test_set = ImgDataset(testdata,transform = test_transform)
test_loader = DataLoader(test_set, batch_size = batch_size, shuffle = False)
model.eval()

Classifier(
  (cnn): Sequential(
    (0): Conv2d(1, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(3, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
    (3): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (4): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(2, 2))
    (5): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (6): ReLU()
    (7): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (8): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (9): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (10): ReLU()
    (11): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  )
  (fc): Sequential(
    (0): Linear(in_features=2048, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=128, bias=True)
    (3): R

In [14]:
prediction = []
with torch.no_grad():
    for i,data in enumerate(test_loader):
        test_pred = model(data.cuda())
        test_label = np.argmax(test_pred.cpu().data.numpy(),axis = 1)
        for y in test_label:
            prediction.append(y)

with open("/kaggle/working/predict.csv",'w') as f:
    f.write('ImageId,Label\n')
    for i,y in enumerate(prediction):
        f.write('{},{}\n'.format(i+1,y))